## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
import matplotlib.pyplot as plt

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7gripper, ((0,0,0.01), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.72,1.52,0.01), (0.25,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.6,-0.3,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=True, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.2,-0.3,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=True, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=True)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepFramer

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane)

In [7]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.SPHERE, "grip0", link_name="indy0_tcp", 
                                dims=(0.01,0.01,0.01), center=(0,0,0.14), rpy=(-np.pi/2,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=False, fixed=True, collision=False)

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

In [8]:
# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=False, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepFramer, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [9]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepFrame
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [10]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [11]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
obs_obj = pscene.create_subject(oname="obstacle", gname="obstacle", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", obstacle, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", obstacle, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", obstacle, [0,0,-0.026], [0,0,0])})

In [12]:
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepFrame("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepFrame("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

## Pipeline

In [13]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

In [14]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [15]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

 * Environment: production
   Use a production WSGI server instead.
('floor', 'floor', 0)
 * Debug mode: off


## import planners

In [16]:

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)

checkers_all = [rcheck, gcheck]

# tcheck = TaskClearanceChecker(pscene, gcheck)
# checkers_all = [tcheck, rcheck, gcheck]

# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

## Test pybullet

In [17]:
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/developing/pddlstream'))

In [18]:
from convert_pscene import *
from plan_rnb import *

#### load pybullet, convert scene

In [19]:
connect_notebook(use_gui=True)
urdf_pybullet_path = copy_meshes(gscene)
robot_body, body_names, movable_bodies = pscene_to_pybullet(pscene, urdf_pybullet_path, tool_name="grip0", name_exclude_list=["indy0"])
print('Objects:', body_names)
saver = WorldSaver()

[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
[WARING] non-object subject not implemented for now
('Objects:', {1L: 'goal', 2L: 'floor', 3L: 'wall', 4L: 'obstacle', 5L: 'box1'})


### Test with IK checkers

In [20]:
gtimer = GlobalTimer.instance()
gtimer.reset()

In [21]:
log_wFeas = []
for _ in range(10):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                          movable=movable_bodies, checkers=checkers_all,
                                          tool_name="grip0", tool_link_name="indy0_tcp", mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_wFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q0), ('AtConf', q0), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p0), ('AtPose', 4L, p0), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p0, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p1), ('AtPose', 5L, p1), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p1, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q0), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-approach-pose]
Setting negate=True for stream [test-cfree-traj-pose]

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.201
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g1), inverse-kinematics:(4, p3, #g1)->(#q2, #t73), test-cfree-traj-pose:(#t73, 5, p4)->(), sample-pose:(4, 1)->(#p1), inverse-kinematics:(4, #p1, #g1)->(#q3, #t74), test-cfree-traj-pose:(#t74, 5, p4)->(), test-cfree-traj-pose:(#t73, 4, p3)->(), test-cfree-pose-pose:(4, #p1, 5, p4)->(), test-cfree-approach-pose:(4, p3, #g1, 5, p4)->(), test-cfree-approach-pose:(4, #p1, #g1, 5, p4)->(), plan-free-motion:(q3, #q2)->(#t116), plan-free-motion:(#q3, q3)->(#t117), plan-holding-motion:(#q2, #q3, 4, #g1)->(#t115)]
Action plan (5, 0.000): [move_free(q3, #q2, #t116), pick(4, p3, #g1, #q2, #t73), move_holding(#q2, #q3, 4, #g1, #t115), place(4, #p1, #g1, #q3, #t74), move_free(#q3, q3, #t117)]
iter=0, outs=1) sample-grasp:(4)->[(g2)]
iter=inf, outs=0) inverse-kinematics:(4, p3, g2)->[]
Sampling for up to 0.223 seconds
iter=1, outs=1) sample-grasp:(4)->[(g3)]
iter=inf, outs=1) inverse-k

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.229
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g2), inverse-kinematics:(4, p10, #g2)->(#q10, #t465), test-cfree-traj-pose:(#t465, 4, p10)->(), sample-pose:(4, 1)->(#p4), inverse-kinematics:(4, #p4, #g2)->(#q11, #t466), test-cfree-traj-pose:(#t466, 5, p11)->(), test-cfree-traj-pose:(#t465, 5, p11)->(), test-cfree-pose-pose:(4, #p4, 5, p11)->(), test-cfree-approach-pose:(4, p10, #g2, 5, p11)->(), test-cfree-approach-pose:(4, #p4, #g2, 5, p11)->(), plan-free-motion:(#q11, q7)->(#t508), plan-free-motion:(q7, #q10)->(#t509), plan-holding-motion:(#q10, #q11, 4, #g2)->(#t507)]
Action plan (5, 0.000): [move_free(q7, #q10, #t509), pick(4, p10, #g2, #q10, #t465), move_holding(#q10, #q11, 4, #g2, #t507), place(4, #p4, #g2, #q11, #t466), move_free(#q11, q7, #t508)]
iter=0, outs=1) sample-grasp:(4)->[(g4)]
iter=inf, outs=1) inverse-kinematics:(4, p10, g4)->[(q8, c13)]
iter=inf, outs=1) test-cfree-traj-pose:(c13, 4, p10)->[()]
iter

iter=inf, outs=1) plan-free-motion:(q17, q11)->[(c21)]
iter=inf, outs=1) plan-holding-motion:(q12, q17, 4, g5)->[(c22)]
Summary: {complexity: 2, cost: 0.000, evaluations: 41, iterations: 3, length: 2, run_time: 0.558, sample_time: 0.328, search_time: 0.230, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 373 | p_success: 0.979 | overhead: 0.001
External: sample-grasp | n: 465 | p_success: 0.991 | overhead: 0.004
External: inverse-kinematics | n: 1619 | p_success: 0.212 | overhead: 0.039
External: plan-free-motion | n: 365 | p_success: 0.801 | overhead: 0.569
External: plan-holding-motion | n: 114 | p_success: 1.000 | overhead: 0.063
External: test-cfree-pose-pose | n: 214 | p_success: 1.000 | overhead: 0.000
External: test-cfree-approach-pose | n: 445 | p_success: 1.000 | overhead: 0.001
External: test-cfree-traj-pose | n: 796 | p_success: 0.883 | overhead: 0.001
Wrote: statistics/py2/kuka-tamp.pkl
         704683 function

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.204
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g5), inverse-kinematics:(4, p22, #g5)->(#q16, #t642), test-cfree-traj-pose:(#t642, 5, p23)->(), test-cfree-traj-pose:(#t642, 4, p22)->(), sample-pose:(4, 1)->(#p7), inverse-kinematics:(4, #p7, #g5)->(#q17, #t643), test-cfree-traj-pose:(#t643, 5, p23)->(), test-cfree-pose-pose:(4, #p7, 5, p23)->(), test-cfree-approach-pose:(4, p22, #g5, 5, p23)->(), test-cfree-approach-pose:(4, #p7, #g5, 5, p23)->(), plan-free-motion:(q23, #q16)->(#t684), plan-free-motion:(#q17, q23)->(#t685), plan-holding-motion:(#q16, #q17, 4, #g5)->(#t686)]
Action plan (5, 0.000): [move_free(q23, #q16, #t684), pick(4, p22, #g5, #q16, #t642), move_holding(#q16, #q17, 4, #g5, #t686), place(4, #p7, #g5, #q17, #t643), move_free(#q17, q23, #t685)]
iter=0, outs=1) sample-grasp:(4)->[(g8)]
iter=inf, outs=0) inverse-kinematics:(4, p22, g8)->[]
Sampling for up to 0.207 seconds
iter=1, outs=1) sample-grasp:(4)->[

iter=inf, outs=0) inverse-kinematics:(4, p25, g14)->[]
Sampling while complexity <= 2

Iteration: 4 | Complexity: 2 | Skeletons: 1 | Skeleton Queue: 5 | Disabled: 0 | Evaluations: 31 | Eager Calls: 0 | Cost: inf | Search Time: 0.209 | Sample Time: 0.310 | Total Time: 0.520
Attempt: 1 | Results: 26 | Depth: 0 | Success: False | Time: 0.042
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 2 to 3
Sampling for up to -0.059 seconds
Sampling while complexity <= 3

Iteration: 5 | Complexity: 3 | Skeletons: 1 | Skeleton Queue: 5 | Disabled: 0 | Evaluations: 31 | Eager Calls: 0 | Cost: inf | Search Time: 0.252 | Sample Time: 0.310 | Total Time: 0.563
Attempt: 1 | Results: 32 | Depth: 0 | Success: False | Time: 0.031
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 3 to 4
Sampling for up to -0.027 seconds
Sampling while complexity <= 4
iter=inf, outs=0) inverse-kinematics:(4, p28, g14)->[]

Iteration:

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.181
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g7), sample-pose:(4, 1)->(#p11), inverse-kinematics:(4, #p11, #g7)->(#q28, #t1276), test-cfree-traj-pose:(#t1276, 5, p35)->(), inverse-kinematics:(4, p34, #g7)->(#q27, #t1275), test-cfree-traj-pose:(#t1275, 4, p34)->(), test-cfree-traj-pose:(#t1275, 5, p35)->(), test-cfree-pose-pose:(4, #p11, 5, p35)->(), test-cfree-approach-pose:(4, p34, #g7, 5, p35)->(), test-cfree-approach-pose:(4, #p11, #g7, 5, p35)->(), plan-free-motion:(#q28, q34)->(#t1317), plan-free-motion:(q34, #q27)->(#t1319), plan-holding-motion:(#q27, #q28, 4, #g7)->(#t1318)]
Action plan (5, 0.000): [move_free(q34, #q27, #t1319), pick(4, p34, #g7, #q27, #t1275), move_holding(#q27, #q28, 4, #g7, #t1318), place(4, #p11, #g7, #q28, #t1276), move_free(#q28, q34, #t1317)]
iter=0, outs=1) sample-grasp:(4)->[(g16)]
iter=0, outs=1) sample-pose:(4, 1)->[(p36)]
iter=inf, outs=0) inverse-kinematics:(4, p36, g16)->[]
Samp

iter=inf, outs=0) inverse-kinematics:(4, p40, g21)->[]
iter=inf, outs=0) inverse-kinematics:(4, p41, g21)->[]
iter=inf, outs=0) inverse-kinematics:(4, p42, g21)->[]
Sampling while complexity <= 2

Iteration: 4 | Complexity: 2 | Skeletons: 1 | Skeleton Queue: 6 | Disabled: 0 | Evaluations: 34 | Eager Calls: 0 | Cost: inf | Search Time: 0.252 | Sample Time: 0.256 | Total Time: 0.508
Attempt: 1 | Results: 45 | Depth: 0 | Success: False | Time: 0.022
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 2 to 3
Sampling for up to 0.018 seconds
iter=inf, outs=0) inverse-kinematics:(4, p43, g21)->[]
Sampling while complexity <= 3

Iteration: 5 | Complexity: 3 | Skeletons: 1 | Skeleton Queue: 5 | Disabled: 0 | Evaluations: 34 | Eager Calls: 0 | Cost: inf | Search Time: 0.275 | Sample Time: 0.387 | Total Time: 0.662
Attempt: 1 | Results: 57 | Depth: 1 | Success: False | Time: 0.111
Attempt: 2 | Results: 259 | Depth: 1 | Success: False | Time: 0.323
Att

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.082
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.184
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g9), inverse-kinematics:(4, p49, #g9)->(#q37, #t1807), test-cfree-traj-pose:(#t1807, 4, p49)->(), test-cfree-traj-pose:(#t1807, 5, p50)->(), sample-pose:(4, 1)->(#p15), inverse-kinematics:(4, #p15, #g9)->(#q38, #t1808), test-cfree-traj-pose:(#t1808, 5, p50)->(), test-cfree-pose-pose:(4, #p15, 5, p50)->(), test-cfree-approach-pose:(4, p49, #g9, 5, p50)->(), test-cfree-approach-pose:(4, #p15, #g9, 5, p50)->(), plan-free-motion:(q52, #q37)->(#t1849), plan-free-motion:(#q38, q52)->(#t1851), plan-holding-motion:(#q37, #q38, 4, #g9)->(#t1850)]
Action plan (5, 0.000): [move_free(q52, #q37, #t1849), pick(4, p49, #g9, #q37, #t1807), move_holding(#q37, #q38, 4, #g9, #t1850), place(4, #p15, #g9, #q38, #t1808), move_free(#q38, q52, #t1851)]
iter=0, outs=1) sample-grasp:(4)->[(g22)]
iter=inf, outs=1) inverse-kinematics

In [22]:
print(gtimer)

solve: 	13734.8 ms/10 = 1373.5 ms (392.745/2972.965)
sample_grasps_4: 	169.2 ms/10 = 16.9 ms (14.887/27.65)
ik_fn: 	3213.2 ms/103 = 31.2 ms (0.582/263.105)
check_feas: 	153.4 ms/103 = 1.5 ms (0.572/4.276)
ReachChecker: 	41.8 ms/103 = 0.4 ms (0.183/1.08)
GraspChecker: 	62.6 ms/82 = 0.8 ms (0.526/2.316)
get_stable_4_1: 	17.5 ms/26 = 0.7 ms (0.417/1.337)
free_motion_gen_ori: 	1600.1 ms/24 = 66.7 ms (4.061/245.578)
holding_motion_gen_ori: 	2028.1 ms/14 = 144.9 ms (11.987/413.761)
get_stable_4_2: 	9.8 ms/12 = 0.8 ms (0.359/1.501)
get_stable_4_3: 	0.0 ms/1 = 0.0 ms (0.007/0.007)



In [23]:
print(log_wFeas)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


### Test without IK checkers

In [20]:
gtimer = GlobalTimer.instance()
gtimer.reset()

In [21]:
log_woFeas = []
for _ in range(10):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                          movable=movable_bodies, checkers=[],
                                          tool_name="grip0", tool_link_name="indy0_tcp", mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_woFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q0), ('AtConf', q0), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p0), ('AtPose', 4L, p0), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p0, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p1), ('AtPose', 5L, p1), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p1, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q0), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-traj-pose]
Setting negate=True for stream [test-cfree-pose-pose]
Set

Attempt: 1 | Results: 11 | Depth: 0 | Success: False | Time: 0.018
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 1 to 2

Iteration: 3 | Complexity: 2 | Skeletons: 0 | Skeleton Queue: 0 | Disabled: 0 | Evaluations: 18 | Eager Calls: 0 | Cost: inf | Search Time: 0.036 | Sample Time: 0.000 | Total Time: 0.036
Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.086
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.189
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g1), sample-pose:(4, 1)->(#p1), inverse-kinematics:(4, #p1, #g1)->(#q3, #t106), test-cfree-traj-pose:(#t106, 5, p7)->(), inverse-kinematics:(4, p6, #g1)->(#q2, #t105), test-cfree-traj-pose:(#t105, 4, p6)->(), test-cfree-traj-pose:(#t105, 5, p7)->(), test-cfree-pose-pose:(4, #p1, 5, p7)->(), test-cfree-approach-pose:(4, p6, #g1, 5, p7)->(), test-cfree-approach-pose:(4, #p1, #g1, 5, p7)->(), plan-free-motion:(q5, #q2)->(#t147), plan-free-motion:(#q3, q5)->

iter=inf, outs=0) plan-free-motion:(q5, q16)->[]
iter=inf, outs=1) inverse-kinematics:(4, p10, g14)->[(q18, c15)]
iter=inf, outs=1) test-cfree-traj-pose:(c15, 5, p7)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c7, 4, p10)->[()]
iter=inf, outs=1) test-cfree-pose-pose:(4, p10, 5, p7)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(4, p9, g10, 5, p7)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(4, p10, g14, 5, p7)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(4, p10, g10, 5, p7)->[()]
iter=inf, outs=1) plan-free-motion:(q7, q5)->[(c16)]
iter=inf, outs=1) test-cfree-pose-pose:(4, p8, 5, p7)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(4, p8, g10, 5, p7)->[()]
iter=inf, outs=1) plan-free-motion:(q6, q5)->[(c17)]
iter=inf, outs=0) inverse-kinematics:(4, p15, g14)->[]
iter=inf, outs=0) inverse-kinematics:(4, p13, g14)->[]
iter=5, outs=1) sample-pose:(4, 1)->[(p18)]
iter=inf, outs=0) inverse-kinematics:(4, p18, g12)->[]
iter=inf, outs=0) inverse-kinematics:(4, p18, g9)->[]
it

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.326
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g2), inverse-kinematics:(4, p20, #g2)->(#q9, #t1321), test-cfree-traj-pose:(#t1321, 4, p20)->(), sample-pose:(4, 1)->(#p4), inverse-kinematics:(4, #p4, #g2)->(#q10, #t1322), test-cfree-traj-pose:(#t1322, 5, p21)->(), test-cfree-traj-pose:(#t1321, 5, p21)->(), test-cfree-pose-pose:(4, #p4, 5, p21)->(), test-cfree-approach-pose:(4, p20, #g2, 5, p21)->(), test-cfree-approach-pose:(4, #p4, #g2, 5, p21)->(), plan-free-motion:(q32, #q9)->(#t1363), plan-free-motion:(#q10, q32)->(#t1365), plan-holding-motion:(#q9, #q10, 4, #g2)->(#t1364)]
Action plan (5, 0.000): [move_free(q32, #q9, #t1363), pick(4, p20, #g2, #q9, #t1321), move_holding(#q9, #q10, 4, #g2, #t1364), place(4, #p4, #g2, #q10, #t1322), move_free(#q10, q32, #t1365)]
iter=0, outs=1) sample-grasp:(4)->[(g16)]
iter=inf, outs=0) inverse-kinematics:(4, p20, g16)->[]
Sampling for up to 0.050 seconds
iter=1, outs=1) sample-gra

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.174
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g3), inverse-kinematics:(4, p27, #g3)->(#q17, #t1713), test-cfree-traj-pose:(#t1713, 4, p27)->(), sample-pose:(4, 1)->(#p7), inverse-kinematics:(4, #p7, #g3)->(#q18, #t1714), test-cfree-traj-pose:(#t1714, 5, p28)->(), test-cfree-traj-pose:(#t1713, 5, p28)->(), test-cfree-pose-pose:(4, #p7, 5, p28)->(), test-cfree-approach-pose:(4, p27, #g3, 5, p28)->(), test-cfree-approach-pose:(4, #p7, #g3, 5, p28)->(), plan-free-motion:(q38, #q17)->(#t1755), plan-free-motion:(#q18, q38)->(#t1756), plan-holding-motion:(#q17, #q18, 4, #g3)->(#t1757)]
Action plan (5, 0.000): [move_free(q38, #q17, #t1755), pick(4, p27, #g3, #q17, #t1713), move_holding(#q17, #q18, 4, #g3, #t1757), place(4, #p7, #g3, #q18, #t1714), move_free(#q18, q38, #t1756)]
iter=0, outs=1) sample-grasp:(4)->[(g19)]
iter=inf, outs=0) inverse-kinematics:(4, p27, g19)->[]
Sampling for up to -0.074 seconds
Sampling while comp

...

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.189
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g5), inverse-kinematics:(4, p40, #g5)->(#q27, #t2193), test-cfree-traj-pose:(#t2193, 4, p40)->(), sample-pose:(4, 1)->(#p11), inverse-kinematics:(4, #p11, #g5)->(#q28, #t2194), test-cfree-traj-pose:(#t2194, 5, p41)->(), test-cfree-traj-pose:(#t2193, 5, p41)->(), test-cfree-pose-pose:(4, #p11, 5, p41)->(), test-cfree-approach-pose:(4, p40, #g5, 5, p41)->(), test-cfree-approach-pose:(4, #p11, #g5, 5, p41)->(), plan-free-motion:(#q28, q67)->(#t2235), plan-free-motion:(q67, #q27)->(#t2237), plan-holding-motion:(#q27, #q28, 4, #g5)->(#t2236)]
Action plan (5, 0.000): [move_free(q67, #q27, #t2237), pick(4, p40, #g5, #q27, #t2193), move_holding(#q27, #q28, 4, #g5, #t2236), place(4, #p11, #g5, #q28, #t2194), move_free(#q28, q67, #t2235)]
iter=0, outs=1) sample-grasp:(4)->[(g25)]
iter=inf, outs=0) inverse-kinematics:(4, p40, g25)->[]
Sampling for up to 0.137 seconds
iter=1, outs=1)

iter=inf, outs=0) inverse-kinematics:(4, p43, g29)->[]

Iteration: 6 | Complexity: 4 | Skeletons: 1 | Skeleton Queue: 1 | Disabled: 0 | Evaluations: 21 | Eager Calls: 0 | Cost: inf | Search Time: 0.260 | Sample Time: 0.583 | Total Time: 0.843
Attempt: 1 | Results: 35 | Depth: 0 | Success: False | Time: 0.039
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 4 to 5
Sampling for up to -0.283 seconds
Sampling while complexity <= 5
iter=3, outs=1) sample-grasp:(4)->[(g30)]
iter=inf, outs=0) inverse-kinematics:(4, p43, g30)->[]

Iteration: 7 | Complexity: 5 | Skeletons: 1 | Skeleton Queue: 1 | Disabled: 0 | Evaluations: 22 | Eager Calls: 0 | Cost: inf | Search Time: 0.302 | Sample Time: 0.669 | Total Time: 0.971
Attempt: 1 | Results: 41 | Depth: 0 | Success: False | Time: 0.021
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 5 to 6
Sampling for up to -0.345 seconds
Sampling while complexity <= 6


iter=inf, outs=1) inverse-kinematics:(4, p48, g34)->[(q76, c64)]
iter=inf, outs=1) test-cfree-traj-pose:(c64, 5, p47)->[()]
iter=inf, outs=1) test-cfree-pose-pose:(4, p48, 5, p47)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(4, p46, g34, 5, p47)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(4, p48, g34, 5, p47)->[()]
iter=inf, outs=1) plan-free-motion:(q73, q75)->[(c65)]
iter=inf, outs=1) plan-free-motion:(q76, q73)->[(c66)]
iter=inf, outs=1) plan-holding-motion:(q75, q76, 4, g34)->[(c67)]
Summary: {complexity: 4, cost: 0.000, evaluations: 38, iterations: 5, length: 2, run_time: 1.317, sample_time: 1.070, search_time: 0.247, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 501 | p_success: 0.974 | overhead: 0.001
External: sample-grasp | n: 571 | p_success: 0.993 | overhead: 0.004
External: inverse-kinematics | n: 2111 | p_success: 0.236 | overhead: 0.046
External: plan-free-motion | n: 591 | p_success: 0.782 | ove

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.193
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g9), sample-pose:(4, 1)->(#p15), inverse-kinematics:(4, #p15, #g9)->(#q36, #t2462), test-cfree-traj-pose:(#t2462, 5, p54)->(), inverse-kinematics:(4, p53, #g9)->(#q35, #t2461), test-cfree-traj-pose:(#t2461, 4, p53)->(), test-cfree-traj-pose:(#t2461, 5, p54)->(), test-cfree-pose-pose:(4, #p15, 5, p54)->(), test-cfree-approach-pose:(4, p53, #g9, 5, p54)->(), test-cfree-approach-pose:(4, #p15, #g9, 5, p54)->(), plan-free-motion:(q80, #q35)->(#t2503), plan-free-motion:(#q36, q80)->(#t2505), plan-holding-motion:(#q35, #q36, 4, #g9)->(#t2504)]
Action plan (5, 0.000): [move_free(q80, #q35, #t2503), pick(4, p53, #g9, #q35, #t2461), move_holding(#q35, #q36, 4, #g9, #t2504), place(4, #p15, #g9, #q36, #t2462), move_free(#q36, q80, #t2505)]
iter=0, outs=1) sample-grasp:(4)->[(g38)]
iter=0, outs=1) sample-pose:(4, 1)->[(p55)]
iter=inf, outs=0) inverse-kinematics:(4, p55, g38)->[]
Samp

iter=inf, outs=0) inverse-kinematics:(4, p65, g40)->[]
iter=inf, outs=0) inverse-kinematics:(4, p65, g42)->[]
iter=inf, outs=0) inverse-kinematics:(4, p65, g41)->[]
iter=5, outs=1) sample-grasp:(4)->[(g43)]
iter=inf, outs=0) inverse-kinematics:(4, p55, g43)->[]
iter=inf, outs=0) inverse-kinematics:(4, p58, g43)->[]
iter=inf, outs=0) inverse-kinematics:(4, p63, g43)->[]
iter=inf, outs=0) inverse-kinematics:(4, p65, g43)->[]
iter=inf, outs=0) inverse-kinematics:(4, p56, g43)->[]
iter=inf, outs=0) inverse-kinematics:(4, p57, g43)->[]

Iteration: 10 | Complexity: 6 | Skeletons: 3 | Skeleton Queue: 22 | Disabled: 0 | Evaluations: 50 | Eager Calls: 0 | Cost: inf | Search Time: 1.370 | Sample Time: 2.966 | Total Time: 4.336
Attempt: 1 | Results: 158 | Depth: 0 | Success: False | Time: 0.063
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 6 to 7
Sampling for up to -1.533 seconds
Sampling while complexity <= 7
iter=5, outs=1) sample-pose:(4, 2)->

In [22]:
print(gtimer)

solve: 	27981.9 ms/10 = 2798.2 ms (725.591/6876.639)
sample_grasps_4: 	171.2 ms/10 = 17.1 ms (12.811/29.345)
ik_fn: 	15575.2 ms/191 = 81.5 ms (4.953/300.776)
check_feas: 	91.4 ms/191 = 0.5 ms (0.229/1.322)
get_stable_4_1: 	23.0 ms/32 = 0.7 ms (0.41/1.437)
free_motion_gen_ori: 	2466.6 ms/28 = 88.1 ms (4.109/296.379)
holding_motion_gen_ori: 	1782.2 ms/14 = 127.3 ms (18.839/288.695)
get_stable_4_2: 	13.0 ms/18 = 0.7 ms (0.315/1.697)
get_stable_4_3: 	0.0 ms/1 = 0.0 ms (0.006/0.006)



In [24]:
print(log_woFeas)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## play plan

In [24]:
SIMULATE = False
if (plan is None) or not has_gui():
    disconnect()
else:
    command = postprocess_plan(plan)
    if SIMULATE:
#         wait_for_user('Simulate?')
        command.control()
    else:
#         wait_for_user('Execute?')
        #command.step()
        command.refine(num_steps=10).execute(time_step=0.001)